In [22]:
import pandas as pd
import numpy as np
import json
import warnings
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

#### 선수가 트레이드 직전/직후 속했던 팀의 스탯만 정제

In [23]:
# 2020년대 트레이드 내역 데이터 불러오기
with open("player_moves/trade_list_2020s.json") as f:
    trades = json.load(f)

# 2020년데 지명권 트레이드 결과 데이터 불러오기
draft_tickets = pd.read_csv("player_moves/draft_tickets_2020s.csv", keep_default_na=False)

In [24]:
moves.head(3)

,날짜,항목,팀,선수,비고,포지션,statizId
0,2024-12-30,FA 계약,KIA,임기영,,투수,10909
1,2024-12-20,FA 계약,삼성,류지혁,,내야수,10894
2,2024-12-19,트레이드,KIA,조상우,키움→KIA,투수,11126


In [28]:
for trade in tqdm(trades):
    for x in trade["playerA"] + trade["playerB"]:
        if x["type"] == "money":
            continue

        elif x["type"] == "draft":
            player_id = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "statizId"].item()
            name = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "선수"].item()
            is_draft = True
            if player_id == '':
                continue

        else:
            name = x["name"]
            player_id = x["statizId"]
            is_draft = False

        
        # with open(f"player_moves/{name}_{player_id}_team_history.json") as f:
        #     moves = json.load(f)
        
        with open(f"player_moves/{name}_{player_id}_team_history.json", encoding='utf-8-sig') as f:
            moves = json.load(f)
        print(moves)

100%|██████████| 40/40 [00:00<00:00, 880.56it/s]

{'name': '장영석', 'statizId': '10404', 'team_history': [{'start_date': '2009-01-01', 'team': '키움'}, {'start_date': '2020-02-01', 'team': 'KIA'}]}
{'name': '박준태', 'statizId': '11260', 'team_history': [{'start_date': '2014-01-01', 'team': 'KIA'}, {'start_date': '2020-02-01', 'team': '키움'}]}
{'name': '추재현', 'statizId': '13094', 'team_history': [{'start_date': '2018-01-01', 'team': '키움'}, {'start_date': '2020-04-08', 'team': '롯데'}]}
{'name': '전병우', 'statizId': '11422', 'team_history': [{'start_date': '2015-01-01', 'team': '롯데'}, {'start_date': '2020-04-08', 'team': '키움'}, {'start_date': '2024-01-01', 'team': '삼성'}]}
{'name': '차재용', 'statizId': '11399', 'team_history': [{'start_date': '2015-01-01', 'team': '롯데'}, {'start_date': '2020-04-07', 'team': '키움'}]}
{'name': '이흥련', 'statizId': '11108', 'team_history': [{'start_date': '2013-01-01', 'team': '삼성'}, {'start_date': '2018-09-08', 'team': '두산'}, {'start_date': '2020-05-30', 'team': 'SSG'}, {'start_date': '2021-03-06', 'team': 'SSG'}]}
{'name